# Utilizando OpenRouteService

Vamos a utilizar ahora OpenRoute service para mostrar un caso de uso real

In [ ]:
#Importar librerias

import random
import matplotlib.pyplot as plt
import pandas as pd
from scipy.spatial.distance import cdist
from ortools.linear_solver import pywraplp
import time
import numpy as np
import folium
from folium.plugins import BeautifyIcon
import openrouteservice as ors
import osmnx as ox



In [2]:
# issue https://github.com/microsoft/vscode/issues/266193

from folium import Map
import base64
from IPython.display import IFrame, display

def show_folium_safe(m : Map, height=1000):
    """
    Displays a Folium map in a safe IFrame using Base64 encoding.
    This avoids "Trusted" errors, file path issues, and CSS leakage.
    """
    # 1. Get the raw HTML string of the map
    html_content = m.get_root().render()
    
    # 2. Encode the HTML to base64
    # This allows us to put the entire map "inside" the URL string
    encoded = base64.b64encode(html_content.encode('utf-8')).decode('utf-8')
    
    # 3. Create a Data URI
    data_uri = f"data:text/html;charset=utf-8;base64,{encoded}"
    
    # 4. Display the IFrame
    # We use width='100%' to fill the cell width, but the CSS is trapped inside
    display(IFrame(src=data_uri, width="100%", height=height))

In [3]:
# Mapa alrededor de Rosario

# Coordenadas del Monumento a la Bandera (aprox)
lat_monumento = -32.9478
lon_monumento = -60.6305

m = folium.Map(location=[lat_monumento, lon_monumento], zoom_start=15)

generar puntos aleatorios en rosario

In [ ]:
print("Descargando red vial del Centro de Rosario...")

# Descargar calles a 1500 metros a la redonda del punto
G = ox.graph_from_point((lat_monumento, lon_monumento), dist=1500, network_type='drive')


# 2. Convertir el grafo a GeoDataFrames (nodos y aristas)
nodes, edges = ox.graph_to_gdfs(G)

# 3. Muestreo Aleatorio
# Seleccionamos 100 nodos (intersecciones) al azar de la red vial
# Esto asegura que los puntos existan físicamente en una calle
puntos_reales = nodes.sample(n=100, random_state=42)

# 4. Crear el DataFrame limpio para tu VRP
df_vrp = pd.DataFrame({
    'osmid': puntos_reales.index, # ID oficial de OpenStreetMap (útil para calcular distancias)
    'latitud': puntos_reales.y.values,
    'longitud': puntos_reales.x.values
})

print(f"Se generaron {len(df_vrp)} puntos válidos sobre calles.")
print(df_vrp.head())


Descargando red vial del Centro de Rosario...
Se generaron 100 puntos válidos sobre calles.
        osmid    latitud   longitud
0   257491698 -32.945130 -60.637867
1   257489792 -32.938708 -60.640588
2   257489020 -32.955139 -60.646171
3  3364478903 -32.952454 -60.627765
4   257490114 -32.943646 -60.640368


In [5]:
for idx, row in df_vrp.iterrows():
    folium.Marker(
        location = [row['latitud'], row['longitud']],
        icon=BeautifyIcon(
            icon_shape='marker',
            number=idx,
            border_color='blue',
            text_color='white',
            background_color='blue'
        )
    ).add_to(m)

In [6]:
show_folium_safe(m)

In [7]:
import networkx as nx
import osmnx as ox
import pandas as pd
import numpy as np

G = ox.add_edge_speeds(G) 
G = ox.add_edge_travel_times(G)

In [8]:
# Lista de los IDs de los nodos que vamos a usar
nodos_interes = df_vrp['osmid'].tolist()
n = len(nodos_interes)

# Inicializamos matrices vacías con ceros
matriz_distancia = np.zeros((n, n))
matriz_tiempo = np.zeros((n, n))

print(f"Calculando rutas entre {n} puntos... (esto puede tomar un momento)")

# Iteramos sobre cada origen y destino
for i, origen in enumerate(nodos_interes):
    for j, destino in enumerate(nodos_interes):
        if i == j:
            continue # Distancia a sí mismo es 0
        
        try:
            # Calcular distancia en metros (weight='length')
            dist = nx.shortest_path_length(G, source=origen, target=destino, weight='length')
            matriz_distancia[i][j] = dist
            
            # Calcular tiempo en segundos (weight='travel_time')
            tiempo = nx.shortest_path_length(G, source=origen, target=destino, weight='travel_time')
            matriz_tiempo[i][j] = tiempo
            
        except nx.NetworkXNoPath:
            # Si no hay camino (ej: calle cortada o sentido único imposible), ponemos un valor muy alto
            matriz_distancia[i][j] = 999999
            matriz_tiempo[i][j] = 999999

print("Cálculo terminado.")

Calculando rutas entre 100 puntos... (esto puede tomar un momento)
Cálculo terminado.


In [9]:
# Asumiendo que tu dataframe se llama 'df_vrp' y tiene la columna 'osmid'
ids = df_vrp['osmid'].values

# Creas el DataFrame usando 'ids' para las filas (index) y columnas
df_matrix_dist = pd.DataFrame(matriz_distancia, index=ids, columns=ids)
df_matrix_time = pd.DataFrame(matriz_tiempo, index=ids, columns=ids)

print(df_matrix_dist.iloc[:5, :5])
print(df_matrix_time.iloc[:5, :5])

             257491698    257489792    257489020    3364478903   257490114 
257491698      0.000000  1039.159428  1780.353610  1912.300513   639.883679
257489792   1030.066008     0.000000  2292.284526  2444.739573   665.809449
257489020   1986.606401  2028.515489     0.000000  2242.456727  1629.239740
3364478903  1585.061390  2386.660940  2277.549876     0.000000  1991.587555
257490114    364.256559   887.023520  1629.551591  2145.402615     0.000000
            257491698   257489792   257489020   3364478903  257490114 
257491698     0.000000  100.908349  172.287340  185.435274   65.105353
257489792    95.758976    0.000000  206.305607  178.581317   59.922850
257489020   190.986132  191.084824    0.000000  180.256413  155.150006
3364478903  144.769905  175.022581  204.979489    0.000000  156.326987
257490114    35.836125   83.091077  154.031031  179.578271    0.000000


In [10]:
df_matrix_dist

,257491698,257489792,257489020,3364478903,257490114,257491537,257490219,257490605,257490911,257490685,...,1307963221,257490119,257490915,1307963235,257490744,2042079341,257490767,257489052,257490904,257489158
257491698,0.000000,1039.159428,1780.353610,1912.300513,639.883679,2834.940086,1115.974646,2013.032716,1926.982715,724.299325,...,1311.105020,708.795089,1382.760838,842.057215,2161.587072,1036.057445,1104.445926,612.634475,2444.946443,1033.914374
257489792,1030.066008,0.000000,2292.284526,2444.739573,665.809449,3367.379147,1630.392853,2863.275182,2461.124004,1754.365333,...,271.945592,1222.468630,1917.482633,332.312592,3011.829538,1095.691960,1921.908759,1126.308016,3266.215774,941.668149
257489020,1986.606401,2028.515489,0.000000,2242.456727,1629.239740,2126.582594,917.201574,1296.940524,1716.726714,1603.094151,...,2300.461081,1074.266298,2261.555664,2348.403008,1445.494880,2469.516977,1983.240752,1168.030287,1728.889306,2023.270435
3364478903,1585.061390,2386.660940,2277.549876,0.000000,1991.587555,1464.256437,1637.592839,1751.656156,558.001294,946.050960,...,2658.606531,1752.065415,558.122771,2189.479913,1900.210513,1061.973779,832.182579,1683.394871,1630.427198,2388.082961
257490114,364.256559,887.023520,1629.551591,2145.402615,0.000000,3068.042189,964.583404,2341.552500,2161.787046,1088.555884,...,1158.969112,556.659181,1618.145675,726.053166,2490.106857,920.053396,1400.186078,460.498567,2744.493092,881.778466
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2042079341,636.070242,1326.222223,2293.476942,1360.145940,929.613775,2282.785514,1650.038402,1767.583222,1376.530371,958.496523,...,1598.167814,1260.138536,832.889000,1129.041196,1916.137579,0.000000,826.216800,1163.977922,2170.523814,1326.109182
257490767,752.878811,1780.470810,1980.711311,1320.654663,1383.862362,2243.294237,1337.272772,1454.817592,1337.039094,645.730893,...,2052.416401,947.372905,791.766081,1583.289783,1603.371948,486.276225,0.000000,851.212291,1865.358487,1780.357769
257489052,822.212215,1375.774472,1169.053024,1824.694201,976.498723,2747.333775,504.084837,1925.426404,1839.376404,636.693014,...,1647.720064,96.160614,1295.154527,1184.008822,2073.980761,1378.009052,1016.839615,0.000000,2357.340131,1370.529418
257490904,2362.031766,3389.623765,1963.385700,1306.266249,2993.015317,1180.090445,1840.921545,958.976158,779.832139,1713.050980,...,3661.569356,2515.864354,1335.093146,3192.442738,1107.530514,2095.429180,1609.152955,2460.365246,0.000000,3389.510724


Ahora ya tenemos las matrices para trabajar el TSP

In [32]:
def points_sub(points, i):
    """
    De una lista de numeros sustrae el numero i en una copia nueva.
    Se considera que poinla lista no tiene elementos duplicados (por ser lista de indices) y que i se encuentra en la lista.

    Args:
      points: lista de puntos.
      i: elemento a sustraer de la lista

    Returns:
      new: lista nueva similar a points pero sin el elemento i.
    """
    new = points.copy()
    new.remove(i)

    return new

In [36]:
distancias = df_matrix_dist  #Cambiar si queremos usar tiempos

# Definimos el modelo
modelo_mtz = pywraplp.Solver.CreateSolver('SAT')

list_index_puntos = list(df.index)

# Definimos como punto de partida el index 0
partida = list_index_puntos[0]

# uso n como n-1 ya que arranco de indice 0 en la lista. Por esto en mtz no resto 1 ni en u_i
n = max(list_index_puntos)

# Definimos variables

x_ij={i:{j: modelo_mtz.IntVar(0,1,'x_'+str(i)+'_'+str(j)) for j in points_sub(list_index_puntos, i)} for i in list_index_puntos}
u_i = {i: modelo_mtz.IntVar(1,n,'u_'+str(i)) for i in points_sub(list_index_puntos, partida)}

# Definimos la función objetivo
obj_expr = sum(x_ij[i][j] * distancias[i][j] for i in list_index_puntos for j in points_sub(list_index_puntos, i))


#Restricciones:

#1
for j in list_index_puntos:
    modelo_mtz.Add(sum(x_ij[i][j] for i in points_sub(list_index_puntos, j)) == 1)

#2
for i in list_index_puntos:
    modelo_mtz.Add(sum(x_ij[i][j] for j in points_sub(list_index_puntos, i)) == 1)

#3
for i in points_sub(list_index_puntos, partida):
    for j in points_sub(points_sub(list_index_puntos, partida), i):
        modelo_mtz.Add(u_i[i] - u_i[j] + 1 <= n * (1- x_ij[i][j]))


# Solver
modelo_mtz.Minimize(obj_expr)

In [ ]:
status = modelo_mtz.Solve()

In [ ]:
modelo_mtz.Objective().Value()

#ToDo